## Simple OpenMM Example

Loading PDB, Solvating, Minimizing and short production run

#### authors
- Antonia Mey
- Jan-Hendrik Prinz

This is a simple introduction and example on how to use openmm. More examples can be found on the openmm website under
http://openmm.org

Misc Imports

In [1]:
import time
from sys import stdout

Import OpenMM

In [2]:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *

In [3]:
print ('OpenMM version is : ' + version.full_version)
start_time = time.time()

OpenMM version is : 6.3.0.dev-Unknown


Check available OpenMM platforms

In [4]:
platform = openmm.Platform.getPlatformByName('CUDA')
print ('Platform used is : ' + platform.getName())
print platform

Platform used is : CUDA
<simtk.openmm.openmm.Platform; proxy of <Swig Object of type 'OpenMM::Platform *' at 0x10652b1b0> >


Loading the unsolvated pdb file

In [5]:
pdb = PDBFile('../data/input.pdb')
forcefield = ForceField('amber99sb.xml', 'tip3p.xml')

Adding solvent

In [6]:
modeller = Modeller(pdb.topology, pdb.positions)
modeller.addSolvent(forcefield, model='tip3p', padding=1*nanometer)

Create System

In [7]:
system1 = forcefield.createSystem(modeller.topology, nonbondedMethod=PME)

Minimizing

In [8]:
integrator1 = VerletIntegrator(0.001*picoseconds)
simulation1 = Simulation(modeller.topology, system1, integrator1, platform)
simulation1.context.setPositions(modeller.positions)
simulation1.minimizeEnergy(maxIterations=200)

Simulate

In [9]:
system2 = forcefield.createSystem(modeller.topology, nonbondedMethod=PME, nonbondedCutoff=1.0*nanometer, constraints=HBonds)
integrator2 = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)

Creating simulation context

In [10]:
simulation2 = Simulation(modeller.topology, system2, integrator2) 
simulation2.context.setPositions(simulation1.context.getState(getPositions=True).getPositions())

Minimizing System 

In [11]:
simulation2.minimizeEnergy(maxIterations=25)

Adding Reporters 

In [12]:
simulation2.reporters.append(PDBReporter('output_exercise1.pdb', 5)) 
simulation2.reporters.append(StateDataReporter(stdout, 100, step=True, potentialEnergy=True, temperature=True))

Running simulation

In [13]:
%%time
simulation2.step(1000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
100,-24905.441484,104.002083718
200,-24491.3622441,144.448023954
300,-24254.8811314,178.261835181
400,-23887.5246872,199.763655856
500,-23524.1666682,222.002724012
600,-23242.3643317,231.738077702
700,-23000.5672438,241.867759347
800,-22911.9797696,252.738879434
900,-22681.1618459,245.539832028
1000,-22621.5378603,264.964602195
CPU times: user 7.94 s, sys: 422 ms, total: 8.36 s
Wall time: 12.1 s


In [1]:
import openmmtools
import openpathsampling as paths
import simtk.openmm as omm
import simtk.unit as u

In [2]:
ala = openmmtools.testsystems.AlanineDipeptideExplicit()

In [3]:
template = paths.tools.snapshot_from_pdb("../data/Alanine_solvated.pdb")

In [4]:
engine = paths.OpenMMEngine(
    template = template, 
    system = ala.system, 
    integrator = omm.VerletIntegrator(1.0*u.femtoseconds),
    options = {
        'platform' : 'CUDA'
    })

In [5]:
engine.create()

In [6]:
st = paths.s

AttributeError: 'module' object has no attribute 'storage'